<a href="https://colab.research.google.com/github/pinilDissanayaka/Psychology-RAG-Fusion/blob/main/Notebook_RAG_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.3 MB/s eta 0:00

In [1]:
import os
from langchain_groq import ChatGroq
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.documents import Document

In [2]:
def getLlm():
  os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
  llm=ChatGroq(model_name='llama-3.1-70b-versatile',
              temperature=0.6)
  return llm

In [3]:
llm=getLlm()

In [4]:
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
pinecone=Pinecone()

In [9]:
pineconeIndexNames=pinecone.list_indexes().names()

print('Available Indexes  :')
print(f'\t {pineconeIndexNames}')

Available Indexes  :
	 ['multi-rag', 'constitution']


In [10]:
INDEX_NAME='rag-fusion'
DIMENSIONS=512

if not INDEX_NAME in pineconeIndexNames:
  pinecone.create_index(
          name=INDEX_NAME,
          dimension=DIMENSIONS,
          metric="euclidean",
          spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
          )
)
else:
  print("Index already exists")

In [11]:
pinecone.describe_index(INDEX_NAME)

{
    "name": "rag-fusion",
    "dimension": 512,
    "metric": "euclidean",
    "host": "rag-fusion-4myrn7y.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [12]:
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question:

{question}"""


multiQyeryPrompt = ChatPromptTemplate.from_template(template)

print(multiQyeryPrompt)

input_variables=['question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five\ndifferent versions of the given user question to retrieve relevant documents from a vector\ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions separated by newlines. Original question:\n\n{question}'))]


In [13]:
def separateQuestions(question):
  return question.split('\n')

In [14]:
multiQyeryChain=multiQyeryPrompt | llm | StrOutputParser() | RunnableLambda(separateQuestions)

In [91]:
multiQyeryChain.invoke('Why Psychologists Rely on Empirical Methods')

['What role do empirical methods play in the field of psychology?',
 'How do psychologists use empirical evidence to support their theories?',
 'What is the importance of empirical research in psychology?',
 'Why is empirical methodology essential in psychological studies?',
 'What are the benefits of using empirical methods in psychological research?']

In [16]:
from urllib import request

file_name="Introducing Psychology"
url="https://ocw.mit.edu/ans7870/9/9.00SC/MIT9_00SCF11_text.pdf"

request.urlretrieve(url, file_name)

('Introducing Psychology', <http.client.HTTPMessage at 0x7d8aeacc6620>)

In [17]:
loader=PyPDFLoader(file_name)

data=loader.load()

In [18]:
pageContent=list()

for page_content in data:
  pageContent.append(page_content.page_content)

print(f'Total pages: {len(pageContent)}')

Total pages: 783


In [26]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [27]:
chunker=SemanticChunker(embeddings)

In [28]:
docs=chunker.split_documents(data)

In [29]:
print(f'Total documents: {len(docs)}')

Total documents: 1969


In [30]:
vectoreStore=PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=INDEX_NAME,
    text_key='text'
)

In [33]:
retriever=vectoreStore.as_retriever()

In [85]:
def reciprocal_rank_fusion(results, k=60):
  fusion_score={}

  for result in results:
    for rank, doc in enumerate(result):
      if doc.page_content not in fusion_score:
        fusion_score[doc.page_content]=0
      else:
        previous_score=fusion_score[doc.page_content]
        new_score=1/(rank+k)
        fusion_score[doc.page_content]=previous_score+new_score

  re_ranked_docs= [(Document(page_content=doc), score) for doc, score in sorted(fusion_score.items(), key=lambda x: x[1], reverse=True)]

  return re_ranked_docs

In [92]:
retrieval_chain_rag_fusion = multiQyeryChain | retriever.map() | RunnableLambda(reciprocal_rank_fusion)
docs = retrieval_chain_rag_fusion.invoke({"question": "Why Psychologists Rely on Empirical Methods"})

len(docs)

8

In [93]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""


qa_prompt = ChatPromptTemplate.from_template(template)

print(qa_prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the following question based on this context:\n\n{context}\n\nQuestion: {question}\n'))]


In [94]:
question="Why Psychologists Rely on Empirical Methods"

In [95]:
final_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

final_chain.invoke({"question":question})

'Psychologists rely on empirical methods because they believe that if their ideas and theories about human behavior are to be taken seriously, they must be backed up by data. Empirical methods allow researchers to collect and analyze data, which helps them to understand the causes of behavior and make predictions about future events. Additionally, empirical methods enable researchers to differentiate between values and facts, and to test hypotheses in a scientific and systematic way.'